In [1]:
import os
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""

In [2]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

import warnings
warnings.filterwarnings('ignore')

In [16]:
path = "Analytics/Customer Projects/OneMain-proj 11012 (Fraud Intel, COIM, RVS, RVO, RV6 Attributes)/File from the Customer/LexisNexis_file.csv"

In [17]:
from dsgtools import azure
data = pd.read_adls(path, reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(data.shape)

(1714348, 29)


In [19]:
freq(data.APPL_STAT)

,Count,Pct,Cuml Count,Cuml Pct
APPL_STAT,,,,
AWTD,369934.0,0.215787,369934.0,0.215787
BOOK,281765.0,0.164357,651699.0,0.380144
DUP,87069.0,0.050788,738768.0,0.430932
PEND,41.0,0.000024,738809.0,0.430956
TUD,31092.0,0.018136,769901.0,0.449093
WTD,944447.0,0.550907,1714348.0,1.000000


In [20]:
freq(data.sample_type)

,Count,Pct,Cuml Count,Cuml Pct
sample_type,,,,
random_sample_app,1705152.0,0.994636,1705152.0,0.994636
straight_roller,9196.0,0.005364,1714348.0,1.000000


In [21]:
data = data[data.APPL_STAT != "DUP"]
print(data.shape)
data = data[data.sample_type == "random_sample_app"]
print(data.shape)
data.APPL_KEY.is_unique

(1627279, 29)
(1618083, 29)


True

## Overall File

In [22]:
data[['P_NM_FRST', 'P_NM_LST', 'P_ADR_LINE1', 'P_ADR_LINE2', 'P_ADR_CITY', 'P_ADR_ST', "P_ADR_ZIP", "P_ADR_ZIP4", 'SSN', 'P_BIRTH_DT', 'primary_phone_number']].isnull().sum()

P_NM_FRST                     2
P_NM_LST                     34
P_ADR_LINE1                3767
P_ADR_LINE2             1520862
P_ADR_CITY                    5
P_ADR_ST                      5
P_ADR_ZIP                     1
P_ADR_ZIP4                93340
SSN                           0
P_BIRTH_DT                   10
primary_phone_number     374208
dtype: int64

In [23]:
data["address"] = np.where(data.P_ADR_LINE1.notnull(), np.where(data.P_ADR_LINE2.notnull(), data.P_ADR_LINE1 + " " + data.P_ADR_LINE2, data.P_ADR_LINE1), data.P_ADR_LINE1)
data["zip"] = np.where(data.P_ADR_ZIP.notnull(), np.where(data.P_ADR_ZIP4.notnull(), data.P_ADR_ZIP + data.P_ADR_ZIP4, data.P_ADR_ZIP), data.P_ADR_ZIP)
Input_FirstName = "P_NM_FRST"
Input_LastName = "P_NM_LST"
Input_StreetAddress = "address"
Input_City = "P_ADR_CITY"
Input_State = "P_ADR_ST"
Input_Zip = "zip"
Input_SSN = "SSN"

data["sufficient_input"] = np.where((data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_Zip].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_City].notnull() & data[Input_State].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_SSN].notnull()), 1, 0)
freq(data["sufficient_input"])

,Count,Pct,Cuml Count,Cuml Pct
sufficient_input,,,,
0,36.0,0.000022,36.0,0.000022
1,1618047.0,0.999978,1618083.0,1.000000


In [25]:
data = data[data["sufficient_input"] == 1]
from datetime import datetime
data["APPDATETIME_fmt"] = pd.to_datetime(data.APPL_ENTRY_DT.str.slice(0,9))
data["APPDATETIME_fmt_F"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7)
data["APPDATETIME_fmt_F_dt"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7) + data['APPDATETIME_fmt'].astype(str).str.slice(8,10)
freq(data["APPDATETIME_fmt_F"] )

,Count,Pct,Cuml Count,Cuml Pct
APPDATETIME_fmt_F,,,,
202106,534383.0,0.330264,534383.0,0.330264
202107,547459.0,0.338346,1081842.0,0.668610
202108,536205.0,0.331390,1618047.0,1.000000


In [26]:
freq(data.delinquent_60at6_ind)

,Count,Pct,Cuml Count,Cuml Pct
delinquent_60at6_ind,,,,
N,256466.0,0.158503,256466.0,0.158503
Y,14123.0,0.008728,270589.0,0.167232
Missing,1347458.0,0.832768,1618047.0,1.000000


In [28]:
data["delinquent_60at6_ind_fmt"] = np.where(data.delinquent_60at6_ind.isnull(), -1, np.where(data.delinquent_60at6_ind  == "Y", 1, 0))
freq(data.delinquent_60at6_ind_fmt)

,Count,Pct,Cuml Count,Cuml Pct
delinquent_60at6_ind_fmt,,,,
-1,1347458.0,0.832768,1347458.0,0.832768
0,256466.0,0.158503,1603924.0,0.991272
1,14123.0,0.008728,1618047.0,1.000000


In [29]:
data = data.sort_values(by = ['P_NM_FRST', 'P_NM_LST', 'address', 'P_ADR_CITY', 'P_ADR_ST', "zip", 'SSN', 'P_BIRTH_DT', 'primary_phone_number',
                              'APPDATETIME_fmt_F_dt', 'delinquent_60at6_ind_fmt'], ascending = False)
print(data.shape)
data = data.drop_duplicates(subset = ['P_NM_FRST', 'P_NM_LST', 'address', 'P_ADR_CITY', 'P_ADR_ST', "zip", 'SSN', 'P_BIRTH_DT', 'primary_phone_number',
                              'APPDATETIME_fmt_F_dt',], keep = "first")
print(data.shape)

(1618047, 36)
(1574646, 36)


In [30]:
1574646-1618047

-43401

In [8]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/OneMain_11012_full_file.parquet",
             overwrite = True)

In [3]:
from dsgtools import azure
data = pd.read_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/OneMain_11012_full_file.parquet",)
print(data.shape)

(1574646, 36)


In [5]:
from datetime import datetime
data["P_BIRTH_DT_fmt"] = pd.to_datetime(data.P_BIRTH_DT.str.slice(0,9), errors = 'coerce', format = "%d%b%Y")
data["P_BIRTH_DT_fmt_dt"] = data['P_BIRTH_DT_fmt'].astype(str).str.slice(0,4) + data['P_BIRTH_DT_fmt'].astype(str).str.slice(5,7) + data['P_BIRTH_DT_fmt'].astype(str).str.slice(8,10)

In [7]:
data[['P_BIRTH_DT', "P_BIRTH_DT_fmt", "P_BIRTH_DT_fmt_dt"]].head()

,P_BIRTH_DT,P_BIRTH_DT_fmt,P_BIRTH_DT_fmt_dt
1355671,26MAR1979,1979-03-26,19790326
1693312,10SEP1996,1996-09-10,19960910
982999,29APR1993,1993-04-29,19930429
649753,03JUN1993,1993-06-03,19930603
1617438,23MAY1998,1998-05-23,19980523


## Sample for validation

In [9]:
data = data[data.APPL_STAT == "BOOK"]
print(data.shape)
freq(data.FRAUD_IND)

(272536, 38)


,Count,Pct,Cuml Count,Cuml Pct
FRAUD_IND,,,,
0,272509.0,0.999901,272509.0,0.999901
1,27.0,0.000099,272536.0,1.000000


In [10]:
data = data[data.FRAUD_IND == "0"]
freq(data.delinquent_60at6_ind_fmt)

,Count,Pct,Cuml Count,Cuml Pct
delinquent_60at6_ind_fmt,,,,
-1,1952.0,0.007163,1952.0,0.007163
0,256442.0,0.941040,258394.0,0.948204
1,14115.0,0.051796,272509.0,1.000000


In [11]:
data = data[data.delinquent_60at6_ind_fmt !=-1]
print(data.shape)

(270557, 38)


In [12]:
# customer_credit_risk_grade
bivariate(data["customer_credit_risk_grade"], data.delinquent_60at6_ind_fmt)

tag                        delinquent_60at6_ind_fmt                     \
stats                                             N      PctN      Sum   
customer_credit_risk_grade                                               
A                                           83942.0  0.310256   4023.0   
B                                           79016.0  0.292049   6631.0   
C                                           31028.0  0.114682   2381.0   
D                                             201.0  0.000743     27.0   
P                                           37118.0  0.137191    752.0   
S                                           39252.0  0.145078    301.0   
Missing                                         0.0  0.000000      0.0   
Total                                      270557.0  1.000000  14115.0   

tag                                                       
stats                           Mean       WoE        IV  
customer_credit_risk_grade                                
A                           0.047926 -0.089355  0.002380  
B                           0.083920  0.509388  0.095511  
C                           0.076737  0.412103  0.023478  
D                           0.134328  1.036413  0.001279  
P                           0.020260 -0.979022  0.086678  
S                           0.007668 -1.963318  0.256342  
Missing                     0.000000  0.000000  0.000000  
Total                       0.052170  0.000000  0.465668

In [13]:
bivariate(data["APPDATETIME_fmt_F"], data.delinquent_60at6_ind_fmt)

tag               delinquent_60at6_ind_fmt                               \
stats                                    N      PctN      Sum      Mean   
APPDATETIME_fmt_F                                                         
202106                             95763.0  0.353948   4984.0  0.052045   
202107                             90928.0  0.336077   4899.0  0.053878   
202108                             83866.0  0.309975   4232.0  0.050461   
Missing                                0.0  0.000000      0.0  0.000000   
Total                             270557.0  1.000000  14115.0  0.052170   

tag                                    
stats                   WoE        IV  
APPDATETIME_fmt_F                      
202106            -0.002564  0.000002  
202107             0.033978  0.000394  
202108            -0.035135  0.000377  
Missing            0.000000  0.000000  
Total              0.000000  0.000773

In [14]:
data = data[~data.customer_credit_risk_grade.isin(["P", "S"])]
print(data.shape)

(194187, 38)


In [15]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/cleaned/OneMain_11_21.parquet", overwrite = True)

In [ ]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/OneMain_11_21.parquet", overwrite = True)